# Знакомство с word2vec


## Загрузка модели
Скачаем модель <code>google-news-vectors</code>. Откроем ее с помощью библиотеки <code>gensim</code>.

In [1]:
! pip install -q -U gensim==3.8.3
! gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM
! pip install -q SciPy==1.5.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (gensim)
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or downloa

In [2]:
! gunzip GoogleNews-vectors-negative300.bin.gz

gzip: GoogleNews-vectors-negative300.bin.gz: No such file or directory


In [3]:
import warnings
warnings.filterwarnings('ignore')

import gensim
from gensim.models import KeyedVectors

w = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin",
                                      binary=True)

ModuleNotFoundError: No module named 'gensim'

Структура называется <code>KeyedVectors</code> и по сути представляет собой отображение между ключами и векторами. Каждый вектор идентифицируется своим ключом поиска, чаще всего коротким строковым токеном, поэтому обычно это соответствие между

<center><code>{str => 1D numpy array}</code></center><br/>



Например, выведем первые 10 координат вектора, соответствующего слову <code>sunrise</code>

In [ ]:
print("Размерность вектора: ", w["sunrise"].shape)
print("Первые 10 координат вектора: \n", w["sunrise"][:10])

## Задание 1. Сходство.

Извлеките векторы слов <code>London</code>, <code>England</code>, <code>Moscow</code>. Посчитайте косинусное расстояние между словами <code>London</code> и <code>England</code> и между словами <code>Moscow</code> и <code>England</code>. Какая пара слов ближе? Подсказка: для вычисления косинусного расстояния использвется метод <code>distance()</code>. Правильный ответ представлен в блоке вывода.

In [ ]:
#enter your code here

## Задание 2. Аналогии.
С помощью метода most_similar решите аналогию
```London : England = Moscow : X```

Правильный ответ представлен в блоке вывода.

(Подсказка: нужно использовать аргументы positive и negative)

In [ ]:
#enter your code here

## Задание 3. Сходство: найти лишнее.
С помощью метода <code>doesnt_match</code> найдите лишнее слово в ряду <code>breakfast cereal dinner lunch</code>.

Правильный ответ представлен в блоке вывода.

In [ ]:
#enter your code here

## Задание 4. Представление предложений в виде векторов


Дано предложение: <code>the quick brown fox jumps over the lazy dog</code>. Вам нужно представить это предложение в виде вектора. Для этого найдите вектор каждого слова в модели, а затем усредните векторы покомпонентно.


In [ ]:
import numpy as np

#enter your code here

# Сравнение двух моделей

## Загрузка ещё одной модели


Откроем модель google-news-vectors и модель, обученную на британском национальном корпусе http://vectors.nlpl.eu/repository/20/0.zip, с помощью gensim.


In [ ]:
! wget -c http://vectors.nlpl.eu/repository/20/0.zip
! unzip 0.zip
! head -3 model.txt

Загрузим модель, обученную на британском национальном корпусе

In [ ]:
w_british = KeyedVectors.load_word2vec_format("model.bin", binary=True)

Заметим, что размерность векторов в этом случае также равна 300. При этом через нижнее подчеркивание нужно указывать часть речи используемого слова. Слова следует приводить к нижнему регистру.

In [ ]:
try:
    print(w_british["London_NOUN"].shape)
    print('upper is ok')
except:
    print(w_british["london_NOUN"].shape)
    print('lower is ok')

## Набор данных для оценки качества
Скачаем датасет wordsim353.



In [ ]:
! wget -c http://alfonseca.org/pubs/ws353simrel.tar.gz
! tar -xvf ws353simrel.tar.gz
! head -5 wordsim353_sim_rel/wordsim_similarity_goldstandard.txt

## Подготовка эталонной выборки


Из файла `wordsim_similarity_goldstandard.txt` извлечем пары слов и посчитаем косинусное сходство их векторов в обеих моделях. Посчитаем корреляцию оценок сходства в модели google-news-vectors с оценками аннотаторов в датасете, а затем - корреляцию сходства в модели на основе британского национального корпуса с оценками аннотаторов в датасете. Какая модель ближе к суждениям экспертов-разметчиков?

(используем только те слова из wordsim, для которых находятся векторы на британском корпусе, помеченные как существительные!)

In [ ]:
import pandas as pd

df = pd.read_csv("wordsim353_sim_rel/wordsim_similarity_goldstandard.txt",
                 sep="\t", header=None)
df.columns = ["first", "second", "score"]
df.head(3)

## Вычисление оценок similarity моделей
Используем только те слова из wordsim, для которых находятся векторы на британском корпусе, помеченные как существительные, сформируйте 3 массива с оценкам схожести:

1. Оценки (косинус между векторами), полученные в результате модели google-news-vectors

2. Оценки (косинус между векторами) полученные в результате модели на основе британского национального корпуса

3. Эталонные оценки из word_sim, для слов из которых находятся векторы на британском корпусе

Пропущенные слова из word_sim представлены в блоке вывода.

In [ ]:
gn_dist, br_dist, scores = [], [], []

for row in df.iterrows():

  w1, w2 = row[1]["first"], row[1]["second"]

  try:

    #enter your code here

  except KeyError as e:
    print(e, "Skipping this word.")

## Выбор модели: корреляция с экспертами

Вычислите корреляцию Спирмена для каждой модели по сравнению с эталонными оценками из word_sim.

Результаты представлены в блоке вывода.

In [ ]:
from scipy.stats import spearmanr

#enter your code here

Можно заметить, что модель google-news-vectors несколько выигрывает в данном случае.